In [0]:
from google.colab import drive
drive.mount('/content/drive')
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import re
import numpy as np
import os
import csv

In [0]:
"""
this function provides information about the array    
"""
def array_inspect(x):
    print ("shape=",(x.shape),"len=",len(x),"ndim=",x.ndim,"size=",x.size,"dtype=",x.dtype)

In [0]:
#np.load("/content/drive/My Drive/Colab Notebooks/tensorflow_basecaller/datasets/transcribed_ref.npy", transcribed_ref)
#np.load("/content/drive/My Drive/Colab Notebooks/tensorflow_basecaller/datasets/x_train2.npy", x_train2)

In [0]:
with open("/content/drive/My Drive/Colab Notebooks/tensorflow_basecaller/datasets/reads-ref.eventalign.csv", mode="r") as csv_file:
  csv_reader = csv.DictReader(csv_file)
  line_count = 0
  for row in csv_reader:
      if line_count == 0:
          print(f'Column names are {", ".join(row)}')
          line_count += 1
      line_count += 1
  print(f'Processed {line_count} lines.')

In [0]:
import pandas
df = pandas.read_csv('/content/drive/My Drive/Colab Notebooks/tensorflow_basecaller/datasets/reads-ref.eventalign.csv')
print(df)

In [0]:
#test to see if its working
for x in range (4):
  print((df['model_kmer'][(x)]), (df['reference_kmer'][(x)]))

In [0]:
def translator(modelout):
  base = str(modelout)
  baseA0 = [re.sub("T","A",str) in base]
  baseT1 = [re.sub("A","T",str) in baseA0]
  baseG2 = [re.sub("G","C",str) in baseT1]
  base_coded = [re.sub("C","G",str) in baseG2] 
  return (baseG2)

In [0]:
reverse = {
		"A" : "T",
		"T" : "A",
		"C" : "G",
		"G" : "C",
		"N" : "N",
		}
print(reverse)
print (reverse.get("A"))
print (reverse.get("T"))
print (reverse.get("G"))
print (reverse.get("C"))

In [0]:
a = df['reference_kmer']
b =np.asarray(a)
f=[]
transcribed_ref=[]
for x in range(len(a)):
  e=str()
  for y in range(len(a[x])):
      b=reverse.get(a[x][y])
      e+=str(b)
  f = np.copy(e)
  transcribed_ref.append(e)
print(transcribed_ref[0])

In [0]:
transcribed_ref = np.load("/content/drive/My Drive/Colab Notebooks/tensorflow_basecaller/datasets/transcribed_ref.npy")
#np.save("/content/drive/My Drive/Colab Notebooks/tensorflow_basecaller/datasets/transcribed_ref.npy", transcribed_ref)

In [0]:
d=np.unique(df['reference_kmer'])
print(len(df['reference_kmer']))
print (len(d))
print(transcribed_ref[0:10])
counts=[]
for x in range(len(d)):
  c=(transcribed_ref.count(d[x]))
  counts.append(c)

plt.subplot()
plt.bar(d, counts)
plt.suptitle('5mer coverage')
plt.show()

In [0]:
#x_train=[df['event_level_mean'], df['event_stdv'], df['event_length'], df['model_mean'], df['model_stdv']]
y_train=df['reference_kmer']
#x_train=np.load("/content/drive/My Drive/Colab Notebooks/tensorflow_basecaller/datasets/x_train2.npy")

In [0]:
#Create a dictionary of the 6mers
#print(y_train) 
released={}
print(d)
for x in range(len(d)):
  a=d[x]
  b=(x)
  released[a] = b



In [0]:
y_train2=[]
for x in range(len(y_train)):
  z=(released.get(y_train[x]))
  y_train2.append(z)
#y_train=np.asarray(y_train2)
#print(y_train2)

In [0]:
print(y_train2[0:10])
y_train=np.asarray(y_train2, dtype=int)
array_inspect(y_train)

In [0]:
#for x in range(len(x_train2)):
#  np.float64(x_train2[(x)])
#array_inspect(x_train2)

#x_train = np.asarray(x_train, dtype = float)
print()
#y_train = np.array()
array_inspect(x_train)
array_inspect(y_train)
#y_train=np.(y_train, dtype = string)
#try enumerating the y_train

In [0]:
x_train2 =[]

for i in range(len(df)):
  
  temp_arr = [df['event_level_mean'][i], df['event_stdv'][i], df['event_length'][i], df['model_mean'][i], df['model_stdv'][i]]  
  x_train2.append(temp_arr)
  if i%100000==0:
    print(i)
x_train2 = np.array(x_train2)

In [0]:
x_train2 = np.load("/content/drive/My Drive/Colab Notebooks/tensorflow_basecaller/datasets/x_train2.npy")
#np.save("/content/drive/My Drive/Colab Notebooks/tensorflow_basecaller/datasets/y_train.npy", y_train)

In [0]:

model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1024, activation=tf.nn.relu)) # uses 128 neurons and is a feed forward rectilinear relu
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu)) # do the same thig for hidden layer 2
model.add(tf.keras.layers.Dense(4096, activation=tf.nn.softmax)) # output layer with number of classifications (256) use softmax for output distribution


In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [0]:
checkpoint_path = "/content/drive/My Drive/Colab Notebooks/tensorflow_basecaller/training_7mer/jochum_nano2.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='loss', 
                                      verbose=1, 
                                      save_weights_only=True, 
                                                
                                      mode='min')
callbacks_list = [checkpoint]
#Create checkpoint callback
#cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
 #                                               save_weights_only=True,
  #                                               verbose=1)

In [0]:
model.load_weights(checkpoint_path)


In [0]:
x_val = x_train2[:800000]
partial_x_train = x_train2[800000:]
y_val = np.array(y_train[0:800000])
partial_y_train = np.array(y_train[800000:])

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=1000,
                    validation_data=(x_val, y_val),
                    batch_size=90000,
                    verbose=1,
                    callbacks = callbacks_list)
model.summary()

In [0]:
results = model.evaluate(x_train2, y_train)

print(results)

In [0]:
min_pred = 1
max_pred = min_pred+1
arr = np.array([x_train2[min_pred][::-1]])

preds = model.predict(x_train2[min_pred:max_pred])
preds2 = model.predict(arr)
print(x_train2[min_pred:max_pred])
print(arr)
print(preds)
print(preds2)
print(np.argmax(preds[0]))
print(np.argmax(preds2[0]))

In [0]:
history_dict = history.history
history_dict.keys()
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [0]:
plt.clf()   # clear figure
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()